![images](https://user-images.githubusercontent.com/42184553/55184987-7a4f1400-51a4-11e9-8476-750bd7160da2.png)

# Immigration Data To The United States
## Data Engineering Capstone Project

### Project Summary

We'll work with three datasets in this project. The main dataset will include data on immigration to the United States, and supplementary datasets will include data on airport codes, U.S. city demographics data.

Our purpose in this project is to equip data for data scientists to assist them with analysis to solve their insights.

**The project follows the follow steps:**
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np 
import datetime as dt
from numpy import int64

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
# Read in the data here
df_airport =  pd.read_csv('airport-codes_csv.csv', encoding = 'utf-8')
df_us_cities =  pd.read_csv('us-cities-demographics.csv', delimiter = ';', encoding = 'utf-8')

## Step 1: Scope the Project and Gather Data

### Scope 
In this project, we will use the data for the analytics table purpose. The data we use is U.S. immigration, and supplementary datasets will include data on airport codes, U.S. city demographics data.

**We want to prepare data for data scientists for analysis to answer these insights:**
- The country from which most immigrants are to the U.S.
- The type of visa issued and the method of transportation most used for immigration.
- The relationship between the volume of immigrants with airports
- Number of immigrants by gender and age
- Number of immigrants by years
- The top five regions to migrate to, and the relationship of the volume of the population to them


**Tools used**

- pandas 
- Apache Spark with SQL

**End Solution**

- Cleaned data for analysis.
- Create the Fact and Dimension tables to ETL pipeline our data.
- Used SQL to check the data quality.


### Describe and Gather Data 

**I94 Immigration Data:** This data comes from the US National Tourism and Trade Office. This is where the data comes from. [Source](https://travel.trade.gov/research/reports/i94/historical/2016.html)



In [4]:
df_immigration.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [5]:
pd.set_option('display.max_columns', 35)
df_immigration.head(2)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1


**Data Dictionary:**

| Column Name | Description                                                                           |
|-------------|---------------------------------------------------------------------------------------|
| CICID       | Unique ID - (float64)                                                                 |
| I94YR       | Year (4 Digit) - (float64)                                                            |
| I94MON      | Month (Numeric) - (float64)                                                           |
| I94CIT      | Source city code for immigration born country (3 digit) - (float64)                   |
| I94RES      | Source country code for immigration residence country (3 digit) - (float64)           |
| I94PORT     | Port addmitted through - (object)                                                     |
| ARRDATE     | Arrival date to the USA - (float64)                                                   |
| I94MODE     | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) - (float64)     |
| I94ADDR     | State of arrival - (object)                                                           |
| DEPDATE     | Departure date - (float64)                                                            |
| I94BIR      | Age of Respondent in Years - (float64)                                                |
| I94VISA     | Visa codes (1=Business, 2=Pleasure, 3=Student) - (float64)                            |
| COUNT       | For summary statistics - (float64)                                                    |
| DTADFILE    | Character Date Field - (int64)                                                        |
| VISAPOST    | Department of State (Visa issued) - (object)                                          |
| OCCUP       | Occupation that will be performed in U.S. - (object)                                  |
| ENTDEPA     | Arrival Flag -  (admitted or paroled into the US) - (object)                          |
| ENTDEPD     | Departure Flag -  (departed, lost visa, or deceased) - (object)                       |
| ENTDEPU     | Update Flag - Update of visa (apprehended, overstayed, or updated to PR) - (float64)  |
| MATFLAG     | Match flag - (object)                                                                 |
| BIRYEAR     | Birth in year (4 Digit) - (float64)                                                   |
| DTADDTO     | Character date field (admitted in the US) - (object)                                  |
| GENDER      | Gender - (object)                                                                     |
| INSNUM      | INS number - (float64)                                                                |
| AIRLINE     | Arrival airline - (object)                                                            |
| ADMNUM      | Admission number (unique and not nullable) - (float64)                                |
| FLTNO       | Flight number of arrival Airline  - (object)                                          |
| VISATYPE    | Class of admission (legally admitting the non-immigrant to temporarily stay in U.S.) - (object)|

**U.S. City Demographic Data:** This data comes from OpenSoft. [Source](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/), it's contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.

This data comes from the US Census Bureau's 2015 American Community Survey.

In [6]:
df_us_cities.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

In [7]:
df_us_cities.head(2)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723


**Data Dictionary:**

| Column Name            | Description                                                                |
|------------------------|----------------------------------------------------------------------------|
| City                   | Name of the city - (object)                                                |
| State                  | US state of the city - (object)                                            |
| Median Age             | The median of the age of the population - (float64)                        |
| Male Population        | Number of the male population - (float64)                                  |
| Female Population      | Number of the female population - (float64)                                |
| Total Population       | Number of the total population - (int64)                                   |
| Number of Veterans     | Number of veterans living in the city - (float64)                          |
| Foreign-born           | Number of residents of the city that were not born in the city - (float64) |
| Average Household Size | Average size of the houses in the city - (float64)                         |
| State Code             | Code of the state of the city - (object)                                   |
| Race                   | Race class - (object)                                                      |
| Count                  | Number of individual of each race - (int64)                                |

**Airport Code Table:** This is a simple table of airport codes and corresponding cities. It comes from here. [Source](https://datahub.io/core/airport-codes#data) 

The airport codes may refer to either IATA airport code, a three-letter code which is used in passenger reservation, ticketing and baggage-handling systems, or the ICAO airport code which is a four letter code used by ATC systems and for airports that do not have an IATA airport code


In [8]:
df_airport.columns

Index(['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country',
       'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code',
       'coordinates'],
      dtype='object')

In [9]:
df_airport.head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"


**Data Dictionary:**

| Column Name  | Description                                       |
|--------------|---------------------------------------------------|
| ident        | Unique identifier - (object)                      |
| type         | Type of the airport - (object)                    |
| name         | Airport Name - (object)                           |
| elevation_ft | Altitude of the airport - (float64)               |
| continent    | Continent - (object)                              |
| iso_country  | ISO code of the country of the airport - (object) |
| iso_region   | ISO code for the region of the airport - (object) |
| municipality | City where the airport is located - (object)      |
| gps_code     | GPS code of the airport - (object)                |
| iata_code    | IATA code of the airport - (object)               |
| local_code   | Local code of the airport - (object)              |
| coordinates  | GPS coordinates of the airport - (object)         |

### Step 2: Explore and Assess the Data
#### Explore the Data 


In [10]:
df_immigration.shape, df_airport.shape ,df_us_cities.shape

((3096313, 28), (55075, 12), (2891, 12))

In [11]:
sum(df_immigration.duplicated()), sum(df_airport.duplicated()), sum(df_us_cities.duplicated())

(0, 0, 0)

**Immigration Database**

In [12]:
pd.set_option('display.max_columns', 35)
df_immigration.head(5)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [13]:
pd.set_option('display.max_columns', 35)
df_immigration.tail(1)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
3096312,3106671.0,2016.0,4.0,123.0,749.0,NOG,20561.0,3.0,AZ,20567.0,58.0,1.0,1.0,20160417,NaN,NaN,Z,O,NaN,M,1958.0,07102016,M,NaN,NaN,5.605687e+10,00866,WB


In [14]:
df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [ ]:
df_immigration.isnull().sum()

In [ ]:
df_immigration['i94mode'].value_counts()

In [ ]:
df_immigration['cicid'].value_counts()

In [ ]:
df_immigration['i94yr'].value_counts()

In [ ]:
df_immigration['i94mon'].value_counts()

**Airport Database**

In [ ]:
df_airport.head(5)

In [ ]:
df_airport.tail(1)

In [ ]:
df_airport.info()

In [ ]:
df_airport.isnull().sum()

In [ ]:
df_airport['iso_country'].value_counts()

**US Cities Database**

City,State, Male Median Age, Population,Female Population,  Total Population, Foreign-born, State Code

In [ ]:
df_us_cities.head(5)

In [ ]:
df_us_cities.tail(1)

In [ ]:
df_us_cities.info()

In [ ]:
df_us_cities.isnull().sum()

In [ ]:
df_us_cities['City'].nunique()

### Cleaning Steps

#### Quality Issues

Now we will define the quality issues in the datasets and clean it 

#### Make copies of the dataframes for cleaning

In [ ]:
# Make copies of the dataframes for cleaning
df_immigration_clean = df_immigration.copy()
df_airport_clean = df_airport.copy()
df_us_cities_clean = df_us_cities.copy()

### 1. Issues

#### Define

In **df_immigration** we will use these columns and the rest we will drop them:  

**cicid, i94yr, i94mon, i94cit, i94res, i94port, i94mode, i94addr, i94bir, i94visa, biryear, gender, airline**

#### Code

In [ ]:
# Drop unused columns
df_immigration_clean.drop(['arrdate', 'depdate', 'count', 'dtadfile','visapost', 
                           'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'dtaddto', 
                           'insnum','admnum', 'fltno', 'visatype'], axis= 1, inplace= True)

#### Test

In [ ]:
df_immigration_clean.columns

### 2. Issues

#### Define

In **df_immigration** in columns **cicid**, **i94yr**, **i94mon**, **i94cit**, **i94res** the types are a **Float** and the data are an intgeres, so we will change as **Integer** type.


#### Code

In [ ]:
df_immigration_clean['cicid'] = df_immigration_clean['cicid'].astype(int64)
df_immigration_clean['i94yr'] = df_immigration_clean['i94yr'].astype(int64)
df_immigration_clean['i94mon'] = df_immigration_clean['i94mon'].astype(int64)
df_immigration_clean['i94cit'] = df_immigration_clean['i94cit'].astype(int64)
df_immigration_clean['i94res'] = df_immigration_clean['i94res'].astype(int64)

#### Test

In [ ]:
df_immigration_clean.head(1)

In [ ]:
df_immigration_clean.info()

### 3. Issues

#### Define

In **df_us_cities** database we will change names labels of columns to **lowercase** and change the space with **"_"**, **"-"** **char**.

#### Code

In [ ]:
df_us_cities_clean.columns= df_us_cities_clean.columns.str.lower()

In [ ]:
df_us_cities_clean.columns= df_us_cities_clean.columns.str.replace(' ', '_')
df_us_cities_clean.columns= df_us_cities_clean.columns.str.replace('-', '_')

#### Test

In [ ]:
df_us_cities_clean.head(1)

### 4. Issues

#### Define

In **df_us_cities** we will use these columns and the rest we will drop them:  

**city, state, median_age, male_population, female_population, total_population, state_code**

#### Code

In [ ]:
# Drop unused columns
df_us_cities_clean.drop(['number_of_veterans', 'foreign_born', 'average_household_size', 
                         'race','count'], axis= 1, inplace= True)

#### Test

In [ ]:
df_us_cities_clean.columns

### 5. Issues

#### Define

In **df_airport** we will use these columns and the rest we will drop them:  

**type, name, elevation_ft, iso_country, iso_region, municipality, local_code**

#### Code

In [ ]:
# Drop unused columns
df_airport_clean.drop(['continent', 'gps_code', 'iata_code', 'coordinates'], axis= 1, inplace= True)

#### Test

In [ ]:
df_airport_clean.columns

### 6. Issues

#### Define

**We will Keep only data for the United State**

#### Code

In [ ]:
df_airport_clean = df_airport_clean[df_airport_clean['iso_country']=='US']

In [ ]:
df_airport_clean.reset_index(drop=True, inplace=True)

#### Test

In [ ]:
df_airport_clean['iso_country'].value_counts()

In [ ]:
df_airport_clean.head(2)

#### Databases & Size

In [ ]:
df_immigration_clean.head(3)

In [ ]:
df_us_cities_clean.head(3)

In [ ]:
df_airport_clean.head(3)

In [ ]:
df_immigration_clean.shape, df_us_cities_clean.shape, df_airport_clean.shape

### Save the cleand dataFrames to csv file

In [ ]:
# Save the cleand dataFrame

df_immigration_clean.to_csv('df_immigration_clean.csv', encoding = 'utf-8', index=False)
df_us_cities_clean.to_csv('df_us_cities_clean.csv', encoding = 'utf-8', index=False)
df_airport_clean.to_csv('df_airport_clean.csv', encoding = 'utf-8', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

Using U.S. immigration, and supplementary datasets, we'll need to create a Star Schema optimized for queries on our insights analysis. This includes the following tables.

### Fact Table
**Fact_immigration** 

cicid, ident, city, state_code,i94yr, i94mon, i94cit, i94res, i94port, i94mode, i94addr, i94bir, i94visa, biryear, gender, airline

### Dimension Tables

**Dim_airport** 

ident, type, name, elevation_ft, iso_country, iso_region, municipality, local_code


**Dim_us_cities**

city, state, median_age, male_population, female_population, total_population, state_code


#### 3.2 Star Schema for U.S. immigration Analysis

![Star_Schema](https://user-images.githubusercontent.com/42184553/91172260-89612b00-e6e4-11ea-9d0d-06eeb51168bc.png)

#### 3.3 Mapping Out Data Pipelines

- Create a Fact table and Dimension tables from the cleaned data.
- Insert data into our tables
- The fact table is created as a SQL query with joins to dimension tables.
- Converted all datasets to a Spark dataframe.
- Written all tabled as final parquet file.
- Run SQL query to check the quality of our tables

### Step 5: Run Pipelines to Model the Data 
#### 5.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Imports Libraries
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [ ]:
# Build spark session
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [ ]:
# Build SQL context object
sqlContext = SQLContext(spark)

In [ ]:
# load the dataset from pandas to spark
df_immigration = spark.read.format("csv").option("header", "true").load("df_immigration_clean.csv")
df_us_cities = spark.read.format("csv").option("header", "true").load("df_us_cities_clean.csv")
df_airport = spark.read.format("csv").option("header", "true").load("df_airport_clean.csv")

In [ ]:
df_immigration.printSchema()

In [ ]:
df_us_cities.printSchema()

In [ ]:
df_airport.printSchema()

**As showing above Spark automatically reads all columns data as strings**

In [ ]:
df_immigration.createOrReplaceTempView("immigration")
df_us_cities.createOrReplaceTempView("us_cities")
df_airport.createOrReplaceTempView("airport")


# Allowing to unlimited time for SQL joins and parquet writes.
sqlContext.setConf("spark.sql.autoBroadcastJoinThreshold", "0")

In [ ]:
# insert data into the us_cities dim table
dim_us_cities = spark.sql(""" SELECT  c.City, 
                                      c.State, 
                                      c.median_age, 
                                      c.male_population, 
                                      c.female_population, 
                                      c.total_population, 
                                      c.state_code
FROM us_cities c
""")

In [ ]:
# insert data into the airport dim table
dim_airport = spark.sql(""" SELECT  a.ident, 
                                        a.type, 
                                        a.name, 
                                        a.elevation_ft, 
                                        a.iso_country, 
                                        a.iso_region, 
                                        a.municipality, 
                                        a.local_code 
FROM airport a
""")

In [ ]:
# insert data into the immigration fact table
fact_immigration = spark.sql(""" SELECT  cicid, 
                                         i94yr, 
                                         i94mon, 
                                         i94cit, 
                                         i94res, 
                                         i94port, 
                                         i94mode, 
                                         i94addr, 
                                         i94bir, 
                                         i94visa, 
                                         biryear, 
                                         gender, 
                                         airline 
FROM immigration 
""")

In [ ]:
fact_immigration.write.parquet("fact_immigration")
dim_airport.write.parquet("dim_airport")
dim_us_cities.write.parquet("dim_us_cities")

#### 5.2 Data Quality Checks


#### Run Quality Checks

In [ ]:
fact_immigration.createOrReplaceTempView("fact_immigration")
dim_airport.createOrReplaceTempView("dim_airport")
dim_us_cities.createOrReplaceTempView("dim_us_cities")

In [ ]:
# this function to check for null values

def nullValue(spark_ctxt, tables_check):  
    for table in tables_check:
        print(f"Data quality check on table {table}...")
        for column in tables_check[table]:
            return_Val = spark_ctxt.sql(f"""SELECT COUNT(*) as nbr FROM {table} WHERE {column} IS NULL""")
            if return_Val.head()[0] > 0:
                raise Value_Error(f"Failed! Found NULL values in {column} column!")
        print(f"Table {table} success! passed.")

In [ ]:
# Tables and Columns to be checked
tables_check = { 'fact_immigration' : ['cicid'], 'dim_us_cities': ['City','state_code'], 'dim_airport':['ident']}
nullValue(spark, tables_check)

In [ ]:
# check the total rows in the table 
spark.sql('SELECT COUNT(*) FROM fact_immigration').show()

In [ ]:
# check the total rows in the table 
spark.sql('SELECT COUNT(*) FROM dim_airport').show()

In [ ]:
# check the total rows in the table 
spark.sql('SELECT COUNT(*) FROM dim_us_cities').show()

In [ ]:
# show 2 rows from our table 
fact_immigration.show(2)

In [ ]:
# show 2 rows from our table 
dim_us_cities.show(2)

In [ ]:
# show 2 rows from our table 
dim_airport.show(2)

In [ ]:
# Distinct combinations of city and state in our dim_us_cities table
spark.sql("""
SELECT COUNT(DISTINCT City, State)
FROM dim_us_cities
""").show()

In [ ]:
# collect data from some columns by 1 entity
spark.sql('''
          SELECT City, male_population, female_population, total_population
          FROM dim_us_cities 
          WHERE City == 'Quincy'
          '''
          ).collect()

#### 5.3 Data dictionary 


**fact_immigration table:**

| Column Name | Description                                                                           |
|-------------|---------------------------------------------------------------------------------------|
| CICID       | Unique ID                                                                 |
| I94YR       | Year                                                          |
| I94MON      | Month                                                          |
| I94CIT      | Source city code for immigration born country                 |
| I94RES      | Source country code for immigration residence country        |
| I94PORT     | Port addmitted through                                               |
| Ident       | Unique identifier - **from Dim_airport**                                                |
| I94MODE     | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)    |
| I94ADDR     | State of arrival                                                          |
| City        | Name of the city - **from Dim_us_cities**                                |
| I94BIR      | Age of Respondent in Years                                              |
| I94VISA     | Visa codes (1=Business, 2=Pleasure, 3=Student)                        |
| BIRYEAR     | Birth in year                                                  |
| GENDER      | Gender                                                                  |
| AIRLINE     | Arrival airline                                                          |

**dim_us_cities table:**

| Column Name            | Description                                                                |
|------------------------|----------------------------------------------------------------------------|
| City                   | Name of the city                                               |
| State                  | US state of the city                                            |
| Median Age             | The median of the age of the population                        |
| Male Population        | Number of the male population                                 |
| Female Population      | Number of the female population                             |
| Total Population       | Number of the total population                                |
| State Code             | Code of the state of the city                                 |

**dim_airport table:**

| Column Name  | Description                                       |
|--------------|---------------------------------------------------|
| ident        | Unique identifier                     |
| type         | Type of the airport                   |
| name         | Airport Name                         |
| elevation_ft | Altitude of the airport              |
| iso_country  | ISO code of the country of the airport |
| iso_region   | ISO code for the region of the airport |
| municipality | City where the airport is located       |
| local_code   | Local code of the airport              |

#### Step 6: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

**First**, I used pandas to load the data and explore and cleand and to save the cleand data for analysis later.

**Second**,  I used Apache Spark with SQL to read, transform, and create data model ETL pipeline,  and to make fact and dimension tables and for check quality.
The reason for this was Spark automatically reads all columns data as strings and the speed of Spark.

**For the approach the problem differently under the following scenarios:**

- As our Insights, The data should be updated every 6 months. 

- We would use Apache Hadoop to create a distributed processing system for faster processing. If the data was increased by 100x, Hadoop can deal with large data.

- We would use Apache Airflow to update on a daily basis that makes us can create a schedule to run a distributed update on all tables with data streamed from the source. And to do that we can store the data on bucket and run a pipeline with S3 on Amazon AWS.

- We would use S3 Redshift and increasing the number of nodes in our cluster on Amazon AWS for increased capacity If the data needs to be accessed by 100+ people..